First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [2]:
model.load_state_dict(torch.load('trt_road_following_circuit_nonormalisation_OK.pth'))

<All keys matched successfully>

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [3]:
from torch2trt import torch2trt

#for RGB 3 channel model
data = torch.zeros((1, 3, 224, 224)).cuda().half()

#for GRAY 1 channel model
#data = torch.zeros((1, 1, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [4]:
torch.save(model_trt.state_dict(), 'trt_road_following_testautomatic3_nonormalisation.pth')

Load the optimized model by executing the cell below

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('trt_road_following_colorIsolation_1channel_OK.pth'))

<All keys matched successfully>

Create the racecar class

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [3]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [6]:
#WITH NORMAL WHEELS MOVING
from utils import preprocess
import numpy as np
import time
liste=[]
STEERING_GAIN = 1.2
STEERING_BIAS = -0.3

car.throttle = 0.0

while True:
    t0=time.time()
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x=output[0]
    t1=time.time()
    timer=t1-t0
    liste.append(timer)
    car.steering = x * STEERING_GAIN + STEERING_BIAS

KeyboardInterrupt: 

In [7]:

print(np.mean(liste))

0.017817944058415823


In [ ]:
#WITH ARCTAN WHEELS MOVING
from utils import preprocess
import numpy as np

STEERING_GAIN = 1.5
STEERING_BIAS = -0.2
STEERING_DGAIN=0.1
angle_last=0.0

car.throttle = 0.25

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x=output[0]
    y=(0.5 - output[1]) / 2.0
    angle=np.arctan2(x,y)
    
    car.steering=angle * STEERING_GAIN + (angle - angle_last) * STEERING_DGAIN + STEERING_BIAS
    angle_last=angle

In [5]:
import ipywidgets
from IPython.display import display

speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [6]:
#WITH WHEELS MOVING
from utils import preprocess
import numpy as np
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    image = preprocess(image).half()
    xy = model_trt(image).detach().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    car.steering = steering_slider.value
    car.throttle = speed_slider.value
    
execute({'new': camera.value})
camera.observe(execute, names='value')